In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

!pip install py-heideltime

from py_heideltime import heideltime

In [4]:
df = pd.read_csv("../Spatial_Annotation_Detection/data/df_sample.csv")
df

,Unnamed: 0,article_id,segment_id,segment_text,annee,entity_location,entity_date,entity_location_cam,entity_date_cam,filtered_entity_location,filtered_entity_location_cam,real_entity_location,real_entity_location_cam,entity_location_gliner,entity_location_camembert,real_entity_location_gliner,real_entity_location_camembert,gold_annotations,gliner_pred_filtered,camembert_pred_filtered
0,0,0,0,Insécurité alimentaire en milieu urbain : La v...,2009,"[""'Ouagadougou'"", ""'Ouagadougou'"", ""'Ouagadoug...","['14 janvier 2009', '14 janvier 2009']","[""'Ouagadougou'"", ""'Ouagadougou'"", ""'Ouagadoug...","['udi 15 janvier 2009 à 02h', 'le 14 janvier 2...","['Kadiogo', 'Kadiogo', 'Kadiogo', 'Kadiogo', '...","['Kadiogo', 'Kadiogo', 'Kadiogo', 'Kadiogo', '...","{""'Ouagadougou'"": {'type': 'city', 'count': 6}}","{""'Ouagadougou'"": {'type': 'city', 'count': 6}...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","{'Ouagadougou': {'type': 'city', 'count': 6}}","{'Ouagadougou': {'type': 'city', 'count': 6}, ...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
1,1,0,1,ont montré que la diversité alimentaire était ...,2009,"[""'Ouagadougou'""]",['2007'],"[""'Ouagadougou'""]",['en 2007.'],['Kadiogo'],['Kadiogo'],"{""'Ouagadougou'"": {'type': 'city', 'count': 1}}","{""'Ouagadougou'"": {'type': 'city', 'count': 1}}","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","{'Ouagadougou': {'type': 'city', 'count': 1}}","{'Ouagadougou': {'type': 'city', 'count': 1}}","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star..."
2,2,1,0,Electrification : Il ny a pas deau pour le cou...,2009,"[""'la Kompienga'"", ""'Bagré'"", ""'Ouagadougou'"",...","['lundi 3 août 2009', 'vendredi 31 juillet 200...","[""'la Kompienga'"", ""'Bagré'"", ""'Centre régiona...","['ndi 3 août 2009 à 02h25min', 'le vendredi 31...","['Boulgou', 'Kadiogo', 'Boulgou']","['Boulgou', 'Kompienga', 'Boulgou']","{""'la Kompienga'"": {'type': 'department', 'cou...","{""'la Kompienga'"": {'type': 'department', 'cou...","[{'label': 'LOC', 'text': 'la Kompienga', 'sta...","[{'label': 'LOC', 'text': 'Kompienga', 'start'...","{'la Kompienga': {'type': 'department', 'count...","{'Kompienga': {'type': 'province', 'count': 1}...","[{'label': 'LOC', 'text': 'Kompienga', 'start'...","[{'label': 'LOC', 'text': 'la Kompienga', 'sta...","[{'label': 'LOC', 'text': 'Kompienga', 'start'..."
3,3,1,1,"actuellement, ne sont pas au mieux de leur tau...",2009,"[""'Bagré'"", ""'Ouagadougou'"", ""'Bagré'"", ""'Komp...",['mi-septembre'],"[""'Bagré'"", ""'CRCO'"", ""'Bobo'"", ""'Ouagadougou'...","['mi-septembre', 'fin de lannée en', 'deux sem...","['Boulgou', 'Kadiogo', 'Boulgou', 'Kompienga',...","['Boulgou', 'Kadiogo', 'Boulgou', 'Kompienga',...","{""'Bagré'"": {'type': 'department', 'count': 2}...","{""'Bagré'"": {'type': 'department', 'count': 2}...","[{'label': 'LOC', 'text': 'Bagré', 'start': 34...","[{'label': 'LOC', 'text': 'Bagré', 'start': 34...","{'Bagré': {'type': 'department', 'count': 2}, ...","{'Bagré': {'type': 'department', 'count': 2}, ...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","[{'label': 'LOC', 'text': 'Bagré', 'start': 34...","[{'label': 'LOC', 'text': 'Bagré', 'start': 34..."
4,4,2,0,Prévision pluviométrique des mois de juillet-a...,2009,"[""'Ouagadougou'"", ""'Ouagadougou'"", ""'laxe Djib...","['vendredi 5 juin 2009', 'mercredi 3 juin 2009...","[""'Ouagadougou'"", ""'Burkina Faso'"", ""'Ouagadou...","['endredi 5 juin 2009 à 02h04min', 'mercredi 3...","['Kadiogo', 'Kadiogo']","['Kadiogo', 'Kadiogo']","{""'Ouagadougou'"": {'type': 'city', 'count': 2}}","{""'Ouagadougou'"": {'type': 'city', 'count': 2}...","[{'label': 'LOC', 'text': 'Ouagadougou', 'star...","[{'label': 'LO

In [4]:
import pandas as pd
import re
from datetime import datetime

# Mapping des mois en chiffres
month_mapping = {
    "janvier": "01", "février": "02", "mars": "03", "avril": "04",
    "mai": "05", "juin": "06", "juillet": "07", "août": "08",
    "septembre": "09", "octobre": "10", "novembre": "11", "décembre": "12"
}


def extract_date(text):
    pattern = r"(lundi|mardi|mercredi|jeudi|vendredi|samedi|dimanche)\s+(\d{1,2}(?:er)?)\s+([a-zéûô]+)\s+(\d{4})\s+à\s+\d{2}h\d{2}min"
    match = re.search(pattern, text, re.IGNORECASE)

    if match:
        day = match.group(2).replace('er', '').zfill(2)
        month_name = match.group(3).lower()
        year = match.group(4)
        month = month_mapping.get(month_name, "00")
        return f"{year}-{month}-{day}"
    return None


article_publication_dates = {}

df = df.copy()


for article_id, group in df.groupby('article_id'):
    first_segment_text = group.iloc[0]['segment_text']  
    publication_date = extract_date(first_segment_text)  
    if publication_date:
        article_publication_dates[article_id] = publication_date
    else:
        article_publication_dates[article_id] = None  


df.loc[:, 'publication_date'] = df['article_id'].map(article_publication_dates)

In [ ]:
df['publication_date'] = df['publication_date'].astype(str)
def extract_heideltime_dates_with_spans(text, dct):

    try:

        timexs = heideltime(
            text,
            language='French',
            document_type='news',
            dct=dct
        )

        results = [
            {

                "date": timex['value'],
                "span": timex['span']
            }
            for timex in timexs if timex['type'] == 'DATE'
        ]
        return results
    except Exception as e:
        print(f"Erreur pour le texte : {text[:30]}... - {e}")
        return []

tqdm.pandas()
df['heideltime_dates_with_spans'] = df.progress_apply(
    lambda row: extract_heideltime_dates_with_spans(row['segment_text'], row['publication_date']),
    axis=1
)

In [ ]:
df.to_csv('./data/new_heaideltime_today.csv', index=False)